In [1]:
from datetime import datetime, date

from pyspark.sql.functions import date_format, upper
from pyspark.sql import SparkSession
from pyspark.sql import Row, Column
import pandas as pd

#### **- pyspark Session 생성**

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/20 07:06:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### **- pyspark DataFrame 생성**
- Session에서 createDataFrame을 이용해 데이터 프레임을 생성할 수 있다.

In [3]:
df = spark.createDataFrame([
    Row(a = 1, b = 2, c = 'str1', d = date(2000, 1, 1), e = datetime(2000, 1, 1, 12, 0)),
    Row(a = 2, b = 3, c = 'str2', d = date(2000, 2, 1), e = datetime(2000, 1, 2, 12, 0)),
    Row(a = 4, b = 5, c = 'str3', d = date(2000, 3, 1), e = datetime(2000, 1, 3, 12, 0)),
])
df

DataFrame[a: bigint, b: bigint, c: string, d: date, e: timestamp]

In [4]:
## pandas Data Frame을 통해서 spark Data Frame으로 생성할 수 있다.
pandas_df = pd.DataFrame({
            'a' : [ 1,  2,  3],
            'b' : [2., 3., 4.],
            'c' : ['str1', 'str2', 'str3'],
            'd' : [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
            'e' : [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)],
        })

df = spark.createDataFrame(pandas_df)
df

/home/utils/anaconda3/envs/ds/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [5]:
df.show()
df.printSchema()

+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|2.0|str1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|str2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|str3|2000-03-01|2000-01-03 12:00:00|
+---+---+----+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [6]:
df.show(1)

+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|2.0|str1|2000-01-01|2000-01-01 12:00:00|
+---+---+----+----------+-------------------+
only showing top 1 row



In [7]:
## spark.sql.repl.eagerEval.enabled를 통해
## pandas DataFrame처럼 표시할 수도 있따.
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2.0,str1,2000-01-01,2000-01-01 12:00:00
2,3.0,str2,2000-02-01,2000-01-02 12:00:00
3,4.0,str3,2000-03-01,2000-01-03 12:00:00


In [8]:
## df.show에서 vertical = True 옵션을 주어
## column이 row가 되도록 표시할 수 있다.
#! df.sample(n).T와 비슷

df.show(2, vertical = True)
pandas_df.sample(2).T

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | str1                
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
-RECORD 1------------------
 a   | 2                   
 b   | 3.0                 
 c   | str2                
 d   | 2000-02-01          
 e   | 2000-01-02 12:00:00 
only showing top 2 rows



,2,0
a,3,1
b,4.0,2.0
c,str3,str1
d,2000-03-01,2000-01-01
e,2000-01-03 12:00:00,2000-01-01 12:00:00


In [9]:
## df.columns로 pyspark DataFrame의 Column도 확인할 수 있따.
## pandas의 df.columns와 동일
print(df.columns)
print(pandas_df.columns)

['a', 'b', 'c', 'd', 'e']
Index(['a', 'b', 'c', 'd', 'e'], dtype='object')


In [10]:
## df.describe를 통해 통계값을 확인할 수 있다.
df.select('a', 'b', 'c').describe().show()

+-------+---+---+----+
|summary|  a|  b|   c|
+-------+---+---+----+
|  count|  3|  3|   3|
|   mean|2.0|3.0|null|
| stddev|1.0|1.0|null|
|    min|  1|2.0|str1|
|    max|  3|4.0|str3|
+-------+---+---+----+



In [11]:
## df.collect()를 통해 데이터 프레임의 row 데이터를 분산 처리할 수 있다.
## 분산 처리를 통해 데이터 프레임이 거대할 경우 메모리 문제를 해결할 수 있다.
df.collect()

[Row(a=1, b=2.0, c='str1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0)),
 Row(a=2, b=3.0, c='str2', d=datetime.date(2000, 2, 1), e=datetime.datetime(2000, 1, 2, 12, 0)),
 Row(a=3, b=4.0, c='str3', d=datetime.date(2000, 3, 1), e=datetime.datetime(2000, 1, 3, 12, 0))]

In [12]:
## df.toPandas()를 통해 pyspark DataFrame을 pandas DataFrame으로 변환할 수도 있다.
#! 데이터 프레임에 datetime 형이 있는 경우 포맷을 변경해줘야 한다.
df.withColumn('e', date_format('e', 'yyyy-MM-dd HH:mm:ss')).toPandas()

,a,b,c,d,e
0,1,2.0,str1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,str2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,str3,2000-03-01,2000-01-03 12:00:00


In [13]:
## pandas 처럼 DataFrame의 column을 instance처럼 이용해 값에 접근할 수 있다.
df.select(df.c).show()

+----+
|   c|
+----+
|str1|
|str2|
|str3|
+----+



In [14]:
## df.withColumn 메소드를 이용해 새로운 columns을 생성할 수 있다.
df.withColumn('upper_c', upper(df.c)).show()

+---+---+----+----------+-------------------+-------+
|  a|  b|   c|         d|                  e|upper_c|
+---+---+----+----------+-------------------+-------+
|  1|2.0|str1|2000-01-01|2000-01-01 12:00:00|   STR1|
|  2|3.0|str2|2000-02-01|2000-01-02 12:00:00|   STR2|
|  3|4.0|str3|2000-03-01|2000-01-03 12:00:00|   STR3|
+---+---+----+----------+-------------------+-------+

